### Librerías necesarias

In [1]:
import pandas as pd
import os
from geopy.distance import geodesic
from tqdm import tqdm

### Funciones

- Paradas cercanas.

In [2]:
def paradas_cercanas_a_validacion(lat_val, lon_val, paradas_linea, umbral):
    punto_validacion = (lat_val, lon_val)
    paradas_cercanas = []

    for _, row in paradas_linea.iterrows():
        coord_parada = (row["LATITUD"], row["LONGITUD"])
        distancia = geodesic(coord_parada, punto_validacion).meters
        if distancia <= umbral:
            paradas_cercanas.append({
                "CodigoParada": int(row["CodigoParada"]),
                "SECUENCIA_GLOBAL": int(row["SECUENCIA_GLOBAL"]),
                "distancia": distancia
            })

    return pd.DataFrame(paradas_cercanas)

- Calcular el tiempo que se demora entre la validación y las paradas candidatas.

In [3]:
def calcular_tiempos_para_candidatas(codigo_linea, secuencia_origen, paradas_candidatas, df_tiempos_prom):
    resultados = []
    df_tiempos_prom["CodigoLinea"] = df_tiempos_prom["CodigoLinea"].astype(int)
    df_tiempos_prom["SECUENCIA_GLOBAL"] = df_tiempos_prom["SECUENCIA_GLOBAL"].astype(int)

    secuencia_origen = secuencia_origen
    
    for _, parada in paradas_candidatas.iterrows():
        t_i = 0.0
        sec_candidata = parada["SECUENCIA_GLOBAL"]
        #print(f" sec_candidata = {sec_candidata}, secuencia_origen = {secuencia_origen}")

        if pd.isna(sec_candidata) or secuencia_origen == sec_candidata:
            #print(" Salta cálculo: secuencia candidata nula o igual al origen")
            t_i = 0.0
        else:
            #print(" Entrando a cálculo de t_i")
            sec_min = min(secuencia_origen, sec_candidata)
            sec_max = max(secuencia_origen, sec_candidata)
            #print(f" Filtrando tiempos entre secuencia {sec_min} y {sec_max}, línea {codigo_linea}")

            tramos = df_tiempos_prom[
                (df_tiempos_prom["CodigoLinea"] == int(codigo_linea)) &
                (df_tiempos_prom["SECUENCIA_GLOBAL"] >= int(sec_min)) &
                (df_tiempos_prom["SECUENCIA_GLOBAL"] <= int(sec_max))
            ]
            #print(f" Tramos encontrados: {len(tramos)}")
            #print(tramos)

            t_i = tramos["tiempo_medio_min"].sum()
            #print(f" t_i final = {t_i}")

        parada_resultado = parada.copy()
        parada_resultado["t_i"] = t_i
        resultados.append(parada_resultado)

    resultados = pd.DataFrame(resultados)
    resultados["CodigoParada"] = resultados["CodigoParada"].astype(int)
    resultados["SECUENCIA_GLOBAL"] = resultados["SECUENCIA_GLOBAL"].astype(int)
    return pd.DataFrame(resultados)

- Inferir las paradas de descenso

In [4]:
def estimar_descensos_usuario(df_afc, df_ejemplo, tiempos_promedio, fw=2, sw=1.2, umbral=500):
    resultados = []

    df_afc = df_afc.copy()
    df_afc["CodigoLinea"] = df_afc["CodigoLinea"].astype(int)
    df_afc["SECUENCIA_GLOBAL"] = df_afc["SECUENCIA_GLOBAL"].astype(int)
    df_afc["CodigoParada"] = df_afc["CodigoParada"].astype(int)

    df_ejemplo.sort_values(by=["CodigoTarjeta", "Tiempo"], inplace=True)

    for tarjeta, grupo in tqdm(df_ejemplo.groupby("CodigoTarjeta"), desc="Procesando tarjetas"):
        grupo = grupo.sort_values("Tiempo").reset_index(drop=True)

        for i in range(len(grupo) - 1):
            actual = grupo.iloc[i]
            siguiente = grupo.iloc[i + 1]

            linea = int(actual["CodigoLinea"])
            secuencia_origen = int(actual["SECUENCIA_GLOBAL"])
            coord_siguiente = (siguiente["LATITUD"], siguiente["LONGITUD"])
            #print(linea)

            paradas_linea = df_afc[
                df_afc["CodigoLinea"] == linea
            ][["CodigoParada", "LATITUD", "LONGITUD", "SECUENCIA_GLOBAL"]].drop_duplicates()

            candidatas = paradas_cercanas_a_validacion(
                coord_siguiente[0],
                coord_siguiente[1],
                paradas_linea,
                umbral=umbral
            )
            #print(candidatas)

            if candidatas.empty:
                resultados.append({
                    "CodigoTarjeta": tarjeta,
                    "ParadaEstimDescenso": "No se pudo estimar",
                    "CodigoLinea": linea,
                    "ParadaInicial": actual["CodigoParada"],
                    "ParadaSiguiente": siguiente["CodigoParada"],
                    "TiempoInicial": actual["Tiempo"],
                    "TiempoSiguiente": siguiente["Tiempo"]
                })
                continue

            candidatas_con_ti = calcular_tiempos_para_candidatas(
                linea,
                secuencia_origen,
                candidatas,
                tiempos_promedio
            )

            candidatas_con_ti["Tg"] = candidatas_con_ti["t_i"] + fw * (candidatas_con_ti["distancia"] / sw) / 60

            filtro_invalidos = (candidatas_con_ti["distancia"] > 0) & (candidatas_con_ti["t_i"] == 0)
            candidatas_validas = candidatas_con_ti[~filtro_invalidos]

            if not candidatas_validas.empty:
                idx = candidatas_validas["Tg"].idxmin()
                mejor_parada = candidatas_validas.loc[idx]
            else:
                mejor_parada = None

            #idx = (candidatas_con_ti["t_i"] + fw * (candidatas_con_ti["distancia"] / sw) / 60).idxmin()
            #mejor_parada = candidatas_con_ti.loc[idx] if idx is not None else None

            if mejor_parada is not None and not mejor_parada.empty:
                tg_i = mejor_parada["t_i"] + fw * (mejor_parada["distancia"] / sw) / 60
                resultados.append({
                    "CodigoTarjeta": tarjeta,
                    "ParadaEstimDescenso": mejor_parada["CodigoParada"],
                    "Tg": tg_i,
                    "TiempoBus": mejor_parada["t_i"],
                    "DistanciaCaminata": mejor_parada["distancia"],
                    "CodigoLinea": linea,
                    "ParadaInicial": actual["CodigoParada"],
                    "ParadaSiguiente": siguiente["CodigoParada"],
                    "TiempoInicial": actual["Tiempo"],
                    "TiempoSiguiente": siguiente["Tiempo"]
                })

        if len(grupo) > 1:
            ultimo = grupo.iloc[-1]
            primero = grupo.iloc[0]

            linea = int(ultimo["CodigoLinea"])
            secuencia_origen = int(ultimo["SECUENCIA_GLOBAL"])
            coord_siguiente = (primero["LATITUD"], primero["LONGITUD"])

            paradas_linea = df_afc[
                df_afc["CodigoLinea"] == linea
            ][["CodigoParada", "LATITUD", "LONGITUD", "SECUENCIA_GLOBAL"]].drop_duplicates()

            candidatas = paradas_cercanas_a_validacion(
                coord_siguiente[0],
                coord_siguiente[1],
                paradas_linea,
                umbral=umbral
            )

            if candidatas.empty:
                resultados.append({
                    "CodigoTarjeta": tarjeta,
                    "ParadaEstimDescenso": "No se pudo estimar (cierre)",
                    "CodigoLinea": linea,
                    "ParadaInicial": ultimo["CodigoParada"],
                    "ParadaSiguiente": primero["CodigoParada"],
                    "TiempoInicial": ultimo["Tiempo"],
                    "TiempoSiguiente": primero["Tiempo"]
                })
                continue

            candidatas_con_ti = calcular_tiempos_para_candidatas(
                linea,
                secuencia_origen,
                candidatas,
                tiempos_promedio
            )

            candidatas_con_ti["Tg"] = candidatas_con_ti["t_i"] + fw * (candidatas_con_ti["distancia"] / sw) / 60

            filtro_invalidos = (candidatas_con_ti["distancia"] > 0) & (candidatas_con_ti["t_i"] == 0)
            candidatas_validas = candidatas_con_ti[~filtro_invalidos]

            if not candidatas_validas.empty:
                idx = candidatas_validas["Tg"].idxmin()
                mejor_parada = candidatas_validas.loc[idx]
            else:
                mejor_parada = None


            #idx = (candidatas_con_ti["t_i"] + fw * (candidatas_con_ti["distancia"] / sw) / 60).idxmin()
            #mejor_parada = candidatas_con_ti.loc[idx] if idx is not None else None

            if mejor_parada is not None and not mejor_parada.empty:
                tg_i = mejor_parada["t_i"] + fw * (mejor_parada["distancia"] / sw) / 60
                resultados.append({
                    "CodigoTarjeta": tarjeta,
                    "ParadaEstimDescenso": mejor_parada["CodigoParada"],
                    "Tg": tg_i,
                    "TiempoBus": mejor_parada["t_i"],
                    "DistanciaCaminata": mejor_parada["distancia"],
                    "CodigoLinea": linea,
                    "ParadaInicial": ultimo["CodigoParada"],
                    "ParadaSiguiente": primero["CodigoParada"],
                    "TiempoInicial": ultimo["Tiempo"],
                    "TiempoSiguiente": primero["Tiempo"]
                })

    return pd.DataFrame(resultados)


### Definir los parámetros para usar la función

In [2]:
afc_25_completo = pd.read_csv("C:/Users/JORGE/OneDrive/Desktop/TIC/Base/afc_25_completo.csv")

- El umbral lo escojemos como el cualtil 90 de la distancia en metros entre las paradas. 

In [3]:
# Parámetros
fw = 2.0
sw = 1.2
umbral = afc_25_completo['distancia_m'].quantile(0.90)

In [4]:
umbral

np.float64(468.17)

- Tiempo promedio

In [7]:
tiempos_promedio = (
    afc_25_completo
    .groupby(['CodigoLinea', 'SECUENCIA_GLOBAL'], as_index=False)
    .agg(tiempo_medio_min=('tiempo_estimado_min', 'mean'))
)

# Seleccionar columnas necesarias para el merge
df_secuencia = afc_25_completo[["CodigoLinea", "SECUENCIA_GLOBAL", "SECUENCIA"]].drop_duplicates()

# Asegurar tipos compatibles
df_secuencia["CodigoLinea"] = df_secuencia["CodigoLinea"].astype(int)
df_secuencia["SECUENCIA_GLOBAL"] = df_secuencia["SECUENCIA_GLOBAL"].astype(int)
tiempos_promedio["CodigoLinea"] = tiempos_promedio["CodigoLinea"].astype(int)
tiempos_promedio["SECUENCIA_GLOBAL"] = tiempos_promedio["SECUENCIA_GLOBAL"].astype(int)

# Merge
tiempos_promedio = tiempos_promedio.merge(df_secuencia, on=["CodigoLinea", "SECUENCIA_GLOBAL"], how="left")

- Ejemplo de aplicación

In [8]:
usuario = "CURA0010274756"
df_ejemplo = afc_25_completo[afc_25_completo["CodigoTarjeta"] == usuario].copy()

In [11]:
resultado_usuario = estimar_descensos_usuario(
    afc_25_completo, 
    df_ejemplo, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

Procesando tarjetas: 100%|██████████| 1/1 [00:00<00:00,  4.60it/s]


In [12]:
resultado_usuario

,CodigoTarjeta,ParadaEstimDescenso,Tg,TiempoBus,DistanciaCaminata,CodigoLinea,ParadaInicial,ParadaSiguiente,TiempoInicial,TiempoSiguiente
0,CURA0010274756,1772.0,0.000000,0.000000,0.000000,2004,1772,1772,14:43:54,14:43:57
1,CURA0010274756,2019.0,13.960663,2.761273,403.178022,2004,1772,4499,14:43:57,18:02:27
2,CURA0010274756,4499.0,0.000000,0.000000,0.000000,5002,4499,4499,18:02:27,18:02:30
3,CURA0010274756,1701.0,23.517826,11.002393,450.555579,5002,4499,1948,18:02:30,18:51:38
4,CURA0010274756,1948.0,0.000000,0.000000,0.000000,6002,1948,1948,18:51:38,18:51:41


### Estimar las paradas de desenso para la base dividida en 1, 2, 3, 4, 5 y 6 validaciones

In [17]:
ruta_carpeta = "C:/Users/JORGE/OneDrive/Desktop/TIC/Inferencia_destinos"
os.makedirs(ruta_carpeta, exist_ok=True)

- Dos validaciones

In [18]:
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo == 2].index
afc_25_2 = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)].copy()

In [19]:
df_descensos_estimados_2 = estimar_descensos_usuario(
    afc_25_completo, 
    afc_25_2, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

Procesando tarjetas: 100%|██████████| 52923/52923 [9:45:35<00:00,  1.51it/s]       


In [20]:
df_descensos_estimados_2.to_csv(os.path.join(ruta_carpeta, "2_val_usuario.csv"), index=False)
df_descensos_estimados_2.to_excel(os.path.join(ruta_carpeta, "descensos_usuario_2.xlsx"), index=False)

- Tres validaciones

In [21]:
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo == 3].index
afc_25_3 = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)].copy()

In [22]:
df_descensos_estimados_3 = estimar_descensos_usuario(
    afc_25_completo, 
    afc_25_3, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

Procesando tarjetas:  32%|███▏      | 6812/21303 [25:37<54:29,  4.43it/s]  


KeyboardInterrupt: 

In [ ]:
df_descensos_estimados_3.to_csv(os.path.join(ruta_carpeta, "3_val_usuario.csv"), index=False)
df_descensos_estimados_3.to_excel(os.path.join(ruta_carpeta, "descensos_usuario_3.xlsx"), index=False)

- Cuatro validaciones

In [ ]:
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo == 4].index
afc_25_4 = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)].copy()

In [ ]:
df_descensos_estimados_4 = estimar_descensos_usuario(
    afc_25_completo, 
    afc_25_4, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

In [ ]:
df_descensos_estimados_4.to_csv(os.path.join(ruta_carpeta, "4_val_usuario.csv"), index=False)
df_descensos_estimados_4.to_excel(os.path.join(ruta_carpeta, "descensos_usuario_4.xlsx"), index=False)

- Cinco validaciones

In [ ]:
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo == 5].index
afc_25_5 = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)].copy()

In [ ]:
df_descensos_estimados_5 = estimar_descensos_usuario(
    afc_25_completo, 
    afc_25_5, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

In [ ]:
df_descensos_estimados_5.to_csv(os.path.join(ruta_carpeta, "5_val_usuario.csv"), index=False)
df_descensos_estimados_5.to_excel(os.path.join(ruta_carpeta, "descensos_usuario_5.xlsx"), index=False)

- Seis validaciones

In [ ]:
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo == 6].index
afc_25_6 = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)].copy()

In [ ]:
df_descensos_estimados_6 = estimar_descensos_usuario(
    afc_25_completo, 
    afc_25_6, 
    tiempos_promedio, 
    fw=2, 
    sw=1.2, 
    umbral=umbral
)

In [ ]:
df_descensos_estimados_6.to_csv(os.path.join(ruta_carpeta, "6_val_usuario.csv"), index=False)
df_descensos_estimados_6.to_excel(os.path.join(ruta_carpeta, "descensos_usuario_6.xlsx"), index=False)